<a href="https://colab.research.google.com/github/roshancharlie/College-Chatbot-Using-ML-and-NLP/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing The Neccesary Libraries

In [1]:
import nltk
import json
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import warnings
warnings.filterwarnings('ignore')

# Creating Query-Based Intents for the Dataset

### Downloading NLP Package

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Processing Data and Synonym Augmentation for Intent Classification

In [4]:
stemmer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)


def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i + 1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences


# Augment the training data using synonym replacement
augmented_data = []
limit_per_tag = 100

for i, doc in enumerate(training):
    bag, output_row = doc
    tokens = [words[j] for j in range(len(words)) if bag[j] == 1]
    augmented_sentences = synonym_replacement(tokens, limit_per_tag)
    for augmented_sentence in augmented_sentences:
        augmented_bag = [1 if augmented_sentence.find(word) >= 0 else 0 for word in words]
        augmented_data.append([augmented_bag, output_row])


combined_data = np.concatenate((training, augmented_data), axis=0)
random.shuffle(combined_data)

# Splitting the Dataset

In [ ]:
from sklearn.model_selection import train_test_split


def separate_data_by_tags(data):
    data_by_tags = {}
    for d in data:
        tag = tuple(d[1])
        if tag not in data_by_tags:
            data_by_tags[tag] = []
        data_by_tags[tag].append(d)
    return data_by_tags.values()


separated_data = separate_data_by_tags(combined_data)

# Lists to store training and testing data
training_data = []
testing_data = []

# Split each tag's data into training and testing sets
for tag_data in separated_data:
    train_data, test_data = train_test_split(tag_data, test_size=0.2, random_state=42)
    training_data.extend(train_data)
    testing_data.extend(test_data)


random.shuffle(training_data)
random.shuffle(testing_data)

# Convert training and testing data back to np.array
train_x = np.array([d[0] for d in training_data])
train_y = np.array([d[1] for d in training_data])
test_x = np.array([d[0] for d in testing_data])
test_y = np.array([d[1] for d in testing_data])

# Training The Model

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        output = self.softmax(x)
        return output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    true_labels = torch.argmax(targets, dim=1)
    correct = (predicted_labels == true_labels).sum().item()
    total = targets.size(0)
    return correct / total

def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = len(test_loader)

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_accuracy += accuracy(outputs, targets) * inputs.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    average_accuracy = total_accuracy / len(test_loader.dataset)
    return average_loss, average_accuracy

In [ ]:
# Create DataLoader for training and testing data
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

batch_size = 64
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model, loss function, and optimizer
input_size = len(train_x[0])
hidden_size = 8
output_size = len(train_y[0])
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model and evaluate on the testing set
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, targets) * inputs.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)

    # Print training loss and accuracy for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")

    # Evaluate on the testing set
    test_loss, test_accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch [1/50], Training Loss: 0.0175, Training Accuracy: 0.3768
Epoch [1/50], Testing Loss: 0.0087, Testing Accuracy: 0.7128
Epoch [2/50], Training Loss: 0.0061, Training Accuracy: 0.8009
Epoch [2/50], Testing Loss: 0.0044, Testing Accuracy: 0.8605
Epoch [3/50], Training Loss: 0.0035, Training Accuracy: 0.8861
Epoch [3/50], Testing Loss: 0.0030, Testing Accuracy: 0.9017
Epoch [4/50], Training Loss: 0.0026, Training Accuracy: 0.9157
Epoch [4/50], Testing Loss: 0.0023, Testing Accuracy: 0.9201
Epoch [5/50], Training Loss: 0.0021, Training Accuracy: 0.9306
Epoch [5/50], Testing Loss: 0.0020, Testing Accuracy: 0.9341
Epoch [6/50], Training Loss: 0.0017, Training Accuracy: 0.9396
Epoch [6/50], Testing Loss: 0.0017, Testing Accuracy: 0.9426
Epoch [7/50], Training Loss: 0.0015, Training Accuracy: 0.9460
Epoch [7/50], Testing Loss: 0.0015, Testing Accuracy: 0.9476
Epoch [8/50], Training Loss: 0.0013, Training Accuracy: 0.9516
Epoch [8/50], Testing Loss: 0.0014, Testing Accuracy: 0.9518
Epoch [9

# Model Inference

In [ ]:
def load_model(model_path, input_size, hidden_size, output_size):
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to preprocess the input sentence
def preprocess_sentence(sentence, words):
    sentence_words = sentence.lower().split()
    sentence_words = [word for word in sentence_words if word in words]
    return sentence_words

# Function to convert the preprocessed sentence into a feature vector
def sentence_to_features(sentence_words, words):
    features = [1 if word in sentence_words else 0 for word in words]
    return torch.tensor(features).float().unsqueeze(0)

# Function to generate a response using the trained model
def generate_response(sentence, model, words, classes):
    sentence_words = preprocess_sentence(sentence, words)
    if len(sentence_words) == 0:
        return "I'm sorry, but I don't understand. Can you please rephrase or provide more information?"

    features = sentence_to_features(sentence_words, words)
    with torch.no_grad():
        outputs = model(features)

    probabilities, predicted_class = torch.max(outputs, dim=1)
    confidence = probabilities.item()
    predicted_tag = classes[predicted_class.item()]

    if confidence > 0.5:
        for intent in intents['intents']:
            if intent['tag'] == predicted_tag:
                return random.choice(intent['responses'])

    return "I'm sorry, but I'm not sure how to respond to that."

In [ ]:
model_path = 'model.pth'
input_size = len(words)
hidden_size = 8
output_size = len(classes)
model = load_model(model_path, input_size, hidden_size, output_size)

# Test the chatbot response
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input, model, words, classes)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.
> Hello
Good to see you!
> what you doing
I'm here to assist you with any questions or information you need. How can I assist you today?
> who are you
You can call me Mind Reader. I'm a Chatbot.
> who created you
I was created by Roshan Sinha.
> quit


In [ ]:
import textwrap

test_cases = [
    "Hello",
    "Good Morning",
    "What are you",
    "Who Created You",
    "Thats not right",
    "What are the class sizes like?",
    "Does your major require an independent study?",
    "Are professors available for research with students?",
    "Are there study abroad programs available?",
    "What's it like to be a first-year student here?",
    "Can students spend a semester abroad?",
    "How can I get involved in research at your college?",
    "Tell me about the extracurricular activities on campus.",
    "What resources are available for career counseling and job placement?",
    "Does the school offer any scholarships or financial aid for international students?",
    "How does the campus support students with disabilities?",
    "Are there any on-campus housing options for upperclassmen?",
    "What opportunities are there for students interested in community service and volunteering?",
    "Is there a mentoring program for freshmen to help with their transition to college life?",
    "What type of on-campus dining options are available?",
    "Tell me about the campus safety measures and security services.",
    "What type of technology and computer facilities are accessible to students?",
    "Are there any notable guest speakers or events that regularly occur on campus?",
    "How does the college foster a sense of community and inclusivity among students?",
    "What is the average student-to-faculty ratio at the university?",
    "Can students participate in sports or fitness activities even if they are not part of a varsity team?",
    "Are there any unique academic programs or centers of excellence at the college?",
    "What type of student organizations are available for students to join?",
    "Tell me about the college's commitment to sustainability and environmental initiatives.",
    "How does the college handle academic support and tutoring for students who need extra help?",
    "Are there any opportunities for students to showcase their artistic talents, such as theater performances or art exhibitions?",
    "Does the college have any partnerships with local businesses or organizations for internships and experiential learning?",
    "What are the on-campus recreational facilities, such as fitness centers or sports fields?",
    "What are the admission requirements for international students?",
    "Can you tell me about the campus facilities, such as the library and student center?",
    "What types of student support services are available, such as counseling and health services?",
    "How diverse is the student body on campus?",
    "What are the top academic programs offered at the college?",
    "Does the college have any special programs for first-generation college students?",
    "What is the college's policy on AP credits and advanced placement?",
    "Are there opportunities for undergraduate students to participate in research with faculty?",
    "What kind of financial aid packages are available for low-income students?",
    "Can you provide information on the college's alumni network and career connections?",
    "Tell me about the college's approach to experiential learning and internships.",
    "What are the campus housing options for students who prefer a single room?",
    "Does the college offer any honors or honors programs for high-achieving students?",
    "What are the popular social events and traditions on campus?",
    "How does the college support students in finding part-time jobs while studying?",
    "Can you tell me more about the college's study abroad partnerships and locations?",
    "What is the college's policy on transfer credits?",
    "Are there any special resources or clubs for students interested in entrepreneurship?",
    "Tell me about the college's sustainability initiatives and green campus efforts.",
    "How accessible are professors for office hours and academic support?",
    "What are the research opportunities available for undergraduate students?",
    "Can you provide information on the college's campus safety statistics and measures?",
    "How does the college celebrate diversity and promote inclusivity on campus?",
    "What are the available options for transportation to and from campus?",
    "Tell me about the college's alumni mentoring programs and career networking events.",
    "Are there any leadership development programs or student government opportunities?",
    "How does the college handle academic advising and course registration for new students?",
    "What are the college's facilities for performing arts and music?",
    "Can you provide information on the college's community service requirements?",
    "Tell me about the college's approach to experiential learning and internships.",
    "What are the campus housing options for students who prefer a single room?",
    "Does the college offer any honors or honors programs for high-achieving students?",
    "What are the popular social events and traditions on campus?",
    "How does the college support students in finding part-time jobs while studying?",
    "Can you tell me more about the college's study abroad partnerships and locations?",
    "What is the college's policy on transfer credits?",
    "Are there any special resources or clubs for students interested in entrepreneurship?",
    "Tell me about the college's sustainability initiatives and green campus efforts.",
    "How accessible are professors for office hours and academic support?",
    "What are the research opportunities available for undergraduate students?",
    "Can you provide information on the college's campus safety statistics and measures?",
    "How does the college celebrate diversity and promote inclusivity on campus?",
    "What are the available options for transportation to and from campus?",
    "Tell me about the college's alumni mentoring programs and career networking events.",
    "Are there any leadership development programs or student government opportunities?",
    "How does the college handle academic advising and course registration for new students?",
    "What are the college's facilities for performing arts and music?",
    "Can you provide information on the college's community service requirements?",
]
print('Testing Cases starting..........\n')

unseen = []
for i in test_cases:
  print(i)
  print('\n')
  text = generate_response(i.lower(),model, words, classes)
  print(textwrap.fill(text, width=70))
  print('-------------------------------------')
  if text == "I'm sorry, but I'm not sure how to respond to that.":
    unseen.append(i)

Testing Cases starting..........

Hello


Good to see you!
-------------------------------------
Good Morning


Good morning! How can I assist you today?
-------------------------------------
What are you


You can call me Mind Reader. I'm a Chatbot.
-------------------------------------
Who Created You


I was created by Roshan Sinha.
-------------------------------------
Thats not right


I'm sorry, but I'm not sure how to respond to that.
-------------------------------------
What are the class sizes like?


I'm sorry, but I'm not sure how to respond to that.
-------------------------------------
Does your major require an independent study?


At our college, students typically have the flexibility to explore
various academic fields during their first year before declaring a
major. The timeline for declaring a major may vary, but academic
advisors are available to guide students through this process.
-------------------------------------
Are professors available for research with st

In [ ]:
unseen

['Thats not right',
 'What are the class sizes like?',
 'Can students spend a semester abroad?',
 'What type of student organizations are available for students to join?',
 'Are there opportunities for undergraduate students to participate in research with faculty?',
 "Can you provide information on the college's campus safety statistics and measures?",
 'What are the available options for transportation to and from campus?',
 "Can you provide information on the college's campus safety statistics and measures?",
 'What are the available options for transportation to and from campus?']